In [2]:
import numpy as np
import librosa
import scipy.signal
from scipy.stats import entropy
import pandas as pd

class VocalTremorAnalyzer:
    def __init__(self, sample_rate=22050):
        self.sr = sample_rate
        
    def load_audio(self, wav_file):
        """WAV 파일을 로드하고 전처리"""
        y, sr = librosa.load(wav_file, sr=self.sr)
        return y
    
    def method1_frequency_variation(self, y, frame_length=2048, hop_length=512):
        """방법 1: 기본 주파수(F0) 변동 분석
        주파수 변화량을 측정하여 떨림 정도를 분석
        """
        # F0 추출
        f0, voiced_flag, voiced_probs = librosa.pyin(y,
                                                    fmin=librosa.note_to_hz('C2'),
                                                    fmax=librosa.note_to_hz('C7'),
                                                    frame_length=frame_length,
                                                    hop_length=hop_length)
        
        # nan 값 제거
        f0 = f0[~np.isnan(f0)]
        
        if len(f0) == 0:
            return {
                'freq_std': 0,
                'freq_variation': 0,
                'tremor_intensity': 0
            }
        
        # 주파수 변동 지표 계산
        freq_std = np.std(f0)  # 주파수 표준편차
        freq_variation = np.abs(np.diff(f0)).mean()  # 주파수 변화량
        tremor_intensity = freq_variation / np.mean(f0) if np.mean(f0) > 0 else 0
        
        return {
            'freq_std': freq_std,
            'freq_variation': freq_variation,
            'tremor_intensity': tremor_intensity
        }
    
    def method2_amplitude_modulation(self, y, frame_length=2048, hop_length=512):
        """방법 2: 진폭 변조 분석
        음성 신호의 포락선(envelope)을 추출하여 진폭 변화를 분석
        """
        # 신호의 포락선 추출
        envelope = np.abs(librosa.stft(y, n_fft=frame_length, hop_length=hop_length))
        envelope_mean = np.mean(envelope, axis=0)
        
        # 진폭 변조 주파수 분석 (4-12Hz 범위의 떨림)
        freqs = librosa.fft_frequencies(sr=self.sr, n_fft=len(envelope_mean))
        fft_envelope = np.abs(np.fft.fft(envelope_mean))
        
        # 4-12Hz 범위의 에너지 계산
        tremor_range = (freqs >= 4) & (freqs <= 12)
        tremor_energy = np.sum(fft_envelope[tremor_range])
        total_energy = np.sum(fft_envelope)
        
        modulation_index = tremor_energy / total_energy if total_energy > 0 else 0
        
        return {
            'modulation_index': modulation_index,
            'tremor_energy': tremor_energy
        }
    
    def method3_spectral_entropy(self, y, frame_length=2048, hop_length=512):
        """방법 3: 스펙트럴 엔트로피 분석
        주파수 도메인에서의 불규칙성을 측정
        """
        # STFT 계산
        D = librosa.stft(y, n_fft=frame_length, hop_length=hop_length)
        magnitude = np.abs(D)
        
        # 각 프레임별 스펙트럴 엔트로피 계산
        spectral_entropy = []
        for frame in magnitude.T:
            prob = frame / np.sum(frame)
            spectral_entropy.append(entropy(prob))
        
        # 엔트로피 변동성 계산
        entropy_std = np.std(spectral_entropy)
        entropy_rate = np.mean(np.abs(np.diff(spectral_entropy)))
        
        return {
            'entropy_std': entropy_std,
            'entropy_rate': entropy_rate
        }
    
    def analyze_wav_files(self, wav_files):
        """여러 WAV 파일의 떨림 분석"""
        results = []
        
        for wav_file in wav_files:
            y = self.load_audio(wav_file)
            
            # 각 방법으로 분석 수행
            freq_analysis = self.method1_frequency_variation(y)
            amp_analysis = self.method2_amplitude_modulation(y)
            entropy_analysis = self.method3_spectral_entropy(y)
            
            # 결과 통합
            result = {
                'file_name': wav_file,
                **freq_analysis,
                **amp_analysis,
                **entropy_analysis
            }
            results.append(result)
        
        return pd.DataFrame(results)